<a href="https://github.com/Filipecdj" rel="some text">
  <img src="https://processodevendas.com/wp-content/uploads/2022/03/PRINCIPAL-FUNDO-ESCURO.png" height="200px" style="display: block; margin: auto;">
</a>


<h2 style="text-align:center">Case Engenheiro de Dados Jr</h2>
<p style="text-align:center;font-size:13px;"></p>
   
<h2 style="text-align:center">Filipe Carvalho de Jesus</h2>
<p style="text-align:center;font-size:13px;">Autor</p>

## Bibliotecas utilizadas para realizar o procedimendo de EL.

In [1]:
import requests
import pandas as pd
from google.cloud import bigquery
from google.oauth2 import service_account
from bs4 import BeautifulSoup

## Headers de configuração para realizar a requisição HTTP.

In [2]:
url = "https://steamdb.info/sales/"
headers = {
    "authority": "steamdb.info",
    "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
    "accept-language": "pt-BR,pt;q=0.9",
    "cache-control": "max-age=0",
    "content-type": "application/x-www-form-urlencoded",
    "cookie": "cf_chl_2=8c153121ae2019c; cf_clearance=MWSSDQfEq0_gJvLhWE9dE2QXeudL9nedwuJLrMUnO1o-1688675358-0-160; __cf_bm=QSkJRA7I.lw5lSox1kpYKCrx7_0.Pr6Z5V1FnSus9WU-1688675363-0-AXa5OP0Kn96BlWN20sfqTEmIkBYAvEtxTMi03Vx7/edtITawhH/j3+G2zPUDR5XpxtKiik7nlJgVnUu3YP3+sjzBPR1e871biIbM38jjAw0J",
    "origin": "https://steamdb.info",
    "referer": "https://steamdb.info/sales/?__cf_chl_tk=Jrs60ajynl93RiacNz2kgOZ_v8e7JJNMU5XD2MtWe5U-1688675358-0-gaNycGzNC6U",
    "sec-ch-ua": '"Not.A/Brand";v="8", "Chromium";v="114", "Google Chrome";v="114"',
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": '"Windows"',
    "sec-fetch-dest": "document",
    "sec-fetch-mode": "navigate",
    "sec-fetch-site": "same-origin",
    "sec-fetch-user": "?1",
    "upgrade-insecure-requests": "1",
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
}

## Realiza o método GET para extrair o HTML do respectivo site.

In [3]:
response = requests.get(url, headers=headers)
if response.status_code == 200:
    html_content = response.content
else:
    print('A solicitação não foi bem-sucedida. Código de status:', response.status_code)

soup = BeautifulSoup(html_content, 'html.parser')

## Cria listas para armazenar os dados extraídos do HTML, bem como listas auxiliares para manipular o dados e obter o resultado desejado.

In [4]:
titulos = []
precos = []
rating = []
release = []
vazio = []
tratado= []
aux = []

## Realiza o tratamento dos dados do HTML:
1. Encontra todos os elementos <td> que possuem a classe "price-discount-major". Esses elementos são armazenados na variável 'elementos'.
2. Em seguida, o código itera sobre cada elemento encontrado na etapa anterior (elementos) e adiciona o texto do elemento pai (i.parent.text) à lista vazio.
3. Após isso, é criado utro laço for, onde itera sobre cada elemento da lista vazio e, em seguida, itera sobre cada linha desse elemento (que é dividido usando a quebra de linha \n como separador).
4. Ao final, a lista aux (que contém as linhas não vazias de um elemento) é adicionada à lista tratado, e em seguida, a lista aux é redefinida como vazia.


In [5]:
elementos = soup.find_all('td', class_='price-discount-major')
for i in elementos:
    vazio.append(i.parent.text)

for j in vazio:
    for p in j.split('\n'):
        if p != '':
            aux.append(p)
    tratado.append(aux)
    aux = []

## Cria o dataframe e insere os dados da lista 'tratado', após isso salva o dataframe em um CSV (steam_sales_data.csv) na pasta do projeto. Ao final, é possivel visualisar a amostra do respectivo dataframe.

In [22]:
df = pd.DataFrame(columns=['Name', 'Highlighted_Deal', 'Discount_Percentage', 'Price', 'Rating', 'Release'])
data_to_append = []

for dado in tratado:
    data_to_append.append({
        'Name': dado[0],
        'Highlighted_Deal': dado[1],
        'Discount_Percentage': dado[2],
        'Price': dado[3],
        'Rating': dado[4],
        'Release': dado[5]
    })

df = pd.concat([df, pd.DataFrame(data_to_append)], ignore_index=True)

df['Discount_Percentage'] = df['Discount_Percentage'].str.replace('%', '').astype(int) / 100
df['Rating'] = df['Rating'].str.replace('%', '').astype(float) / 100

df.to_csv('steam_sales_data.csv', index=False, encoding='utf-8')

styled_df = df.head(10).style.set_properties(**{'max-width': '100px'})
display(styled_df)

,Name,Highlighted_Deal,Discount_Percentage,Price,Rating,Release
0,Drakensang,new historical low,-0.800000,"R$ 3,39",0.757100,Feb 2009
1,Borderlands 2,"all-time low: R$ 11,18",-0.750000,"R$ 11,24",0.927900,Sep 2012
2,Portal 2 - The Final Hours,new historical low,-0.800000,"R$ 0,89",0.737200,May 2011
3,Owlboy,new historical low,-0.650000,"R$ 16,09",0.818000,Nov 2016
4,Really Big Sky,new historical low,-0.850000,"R$ 1,57",0.728700,Feb 2012
5,Distance,new historical low,-0.800000,"R$ 9,49",0.897900,Sep 2018
6,Bridge Constructor,new historical low,-0.900000,"R$ 1,69",0.730500,Oct 2013
7,Livelock,new historical low,-0.850000,"R$ 3,10",0.753800,Aug 2016
8,The Inner World,new historical low,-0.900000,"R$ 2,49",0.856000,Sep 2013
9,SteamWorld Dig,new historical low,-0.900000,"R$ 2,06",0.906100,Dec 2013


## Lê o csv salvo na pasta do projeto e insere no DW do Google Big Query.

In [24]:
steam_data = pd.read_csv('steam_sales_data.csv')
key_path = r"C:\Users\filip\Desktop\python_codigos\GBQ.json"
credentials = service_account.Credentials.from_service_account_file(key_path, scopes=["https://www.googleapis.com/auth/cloud-platform"])
steam_data.to_gbq(credentials=credentials, destination_table='beanalytic.steam_data',if_exists='replace')

100%|██████████| 1/1 [00:00<?, ?it/s]
